# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [6]:
max_page = 8
movies = []
for page in range(1, max_page + 1):
    url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
    payload = {'page': str(page)}
    response = requests.get(url, params=payload)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, "lxml")
    movie_list = soup.find("select", attrs={'name': "movie_id"}).find_all("option", attrs={'data-name': re.compile('.*')})
    for m in movie_list:
        movie = {}
#         movie_name = m['data-name']
#         movie_id = m['value']
        movie['Movie'] = m['data-name']
        movie['id'] = m['value']
        movies.append(movie)
        
for movie in movies:
    print('Movie: ' + movie['Movie'] + ', id: ' + movie['id'])
        

Movie: 紫羅蘭永恆花園電影版, id: 11260
Movie: 末日終結戰, id: 11246
Movie: 竊聽風暴：數位修復版, id: 11234
Movie: FANCY – 以情慾與詩為名, id: 11233
Movie: 詐欺女王, id: 11217
Movie: 銀魂 THE FINAL, id: 11209
Movie: 刺殺小說家, id: 11205
Movie: 穿越時空遇見妳, id: 11204
Movie: 我的母親 經典數位修復, id: 11195
Movie: 100%小狼人, id: 11182
Movie: 複身犯, id: 11160
Movie: 晨曦將至, id: 11121
Movie: 夏目友人帳特別上映版—喚石與可疑訪客, id: 11104
Movie: 澪之料理帖, id: 11098
Movie: 結婚計念日, id: 11208
Movie: 迴路追殺令, id: 11207
Movie: 墮落天使, id: 11197
Movie: 小小的戀愛情事, id: 11193
Movie: 皇家賓館, id: 11151
Movie: 牛宰褲, id: 10939
Movie: 鬼影, id: 11224
Movie: 鬼鄰居, id: 11194
Movie: 偷天行動, id: 11187
Movie: 男兒王, id: 11186
Movie: 跟你老婆去旅行, id: 11183
Movie: STAND BY ME 哆啦A夢2, id: 11179
Movie: 我沒有談的那場戀愛, id: 11159
Movie: 銀龍出任務, id: 11156
Movie: 金不厭詐, id: 11090
Movie: 湯姆貓與傑利鼠, id: 11072
Movie: 角頭–浪流連, id: 11162
Movie: 諾亞方舟大冒險, id: 11155
Movie: 我啊，走自己的路, id: 11088
Movie: T-34：玩命坦克, id: 11178
Movie: 無邪, id: 11168
Movie: 騙騙愛上你, id: 11165
Movie: 龍貓, id: 11163
Movie: 緝魂, id: 11113
Movie: 海豚總動員, id: 11068
Movie: 細

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [11]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 11179

In [12]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
# print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 宜蘭, 代號(area_id): 11


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [13]:
# 指定放映地區
area_id = 28

In [14]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

三月 1
三月 2
三月 3
三月 4
三月 5


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [15]:
# 選定要觀看的日期
date = "2021-03-02"

In [17]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

<div class="pc-movie-schedule-form">
    <div class="area_timebox">
   <div class="area_title">台北市</div>
      <ul id="theater_id_32" class="area_time _c jq_area_time" data-theater_name="台北美麗華大直影城" data-theater_url="http://www.miramarcinemas.tw/" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=32" >
       <li class="adds">
           <a href="https://movies.yahoo.com.tw/theater_result.html/id=32">台北美麗華大直影城</a>
           <span>02-85022208</span>
       </li>
                         <li class="taps">
                <span class="tapR">IMAX</span>                           </li>
           <li class="time _c">
               <div class="input_picker jq_input_picker">
                                                                   <input type="radio" name="schedule_list" id="75048801" class="gabtn"
                        value="2021-03-02 09:40:00"  data-movie="ebd86082ad1642518dccd76e10a4c623"
                        data-cinema="40288eb0050f31210ada0f556

In [22]:
for data in html:
    print('電影院: ' + data['data-theater_name'])
    information = data.find_all('input')
    for i in information:
          print('    放映類型: ' + i['data-movie_type'] + ' 時間: ' + i['value'])
    print()
          


電影院: 台北美麗華大直影城
    放映類型: IMAX 時間: 2021-03-02 09:40:00
    放映類型: IMAX 時間: 2021-03-02 14:00:00
    放映類型: IMAX 時間: 2021-03-02 16:00:00
    放映類型: IMAX 時間: 2021-03-02 20:20:00
    放映類型: IMAX 時間: 2021-03-02 22:20:00
    放映類型: 數位 時間: 2021-03-02 12:10:00
    放映類型: 數位 時間: 2021-03-02 17:45:00

電影院: 台北新光影城
    放映類型: 數位 時間: 2021-03-02 17:00:00

電影院: 喜滿客絕色影城
    放映類型: 數位 時間: 2021-03-02 12:45:00
    放映類型: 數位 時間: 2021-03-02 15:10:00
    放映類型: 數位 時間: 2021-03-02 17:30:00

電影院: 台北信義威秀影城
    放映類型: 數位 時間: 2021-03-02 09:40:00
    放映類型: 數位 時間: 2021-03-02 15:05:00
    放映類型: 數位 時間: 2021-03-02 17:00:00
    放映類型: 數位 時間: 2021-03-02 12:10:00
    放映類型: 數位 時間: 2021-03-02 18:25:00
    放映類型: 數位 時間: 2021-03-02 22:45:00

電影院: 京站威秀影城
    放映類型: 數位 時間: 2021-03-02 09:45:00
    放映類型: 數位 時間: 2021-03-02 14:45:00
    放映類型: 數位 時間: 2021-03-02 12:10:00
    放映類型: 數位 時間: 2021-03-02 16:30:00
    放映類型: 數位 時間: 2021-03-02 21:20:00

電影院: 梅花數位影院
    放映類型: 數位 時間: 2021-03-02 12:25:00

電影院: 台北新民生戲院
    放映類型: 數位 時間: 2021-03-02 18:05:00

電影院: